 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500 Index.

In [ ]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
from pathlib import Path
%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp500_history.csv`: Contains historical closing prices of the S&P 500 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [ ]:
# Reading whale returns
whale_csv_path = (r"C:\Users\minha\OneDrive\Documents\Pandas_Portfolio_Analysis\Resources\whale_returns.csv")
whales_df = pd.read_csv(whale_csv_path)
whales_df.set_index(pd.to_datetime(whales_df["Date"], infer_datetime_format = True), inplace = True)
whales_df.drop(columns = ["Date"], inplace = True)
whales_df.head()

In [ ]:
# Count nulls
whales_df.isnull().sum()

In [ ]:
# Drop nulls
whales_df = whales_df.dropna().copy() 
whales_df

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [ ]:
# Reading algorithmic returns
algo_csv_path = (r"C:\Users\minha\OneDrive\Documents\Pandas_Portfolio_Analysis\Resources\algo_returns.csv")
algo_df = pd.read_csv(algo_csv_path)
algo_df.set_index(pd.to_datetime(algo_df["Date"], infer_datetime_format = True), inplace = True)
algo_df.drop(columns = ["Date"], inplace = True)
algo_df.head()

In [ ]:
# Count nulls
algo_df.isnull().sum()

In [ ]:
# Drop nulls
algo_df = algo_df.dropna().copy() 
algo_df

## S&P 500 Returns

Read the S&P 500 historic closing prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P 500 Closing Prices
sp500_csv_path = (r"C:\Users\minha\OneDrive\Documents\Pandas_Portfolio_Analysis\Resources\sp500_history.csv")
sp500_df = pd.read_csv(sp500_csv_path)
sp500_df.set_index(pd.to_datetime(sp500_df["Date"], infer_datetime_format = True), inplace = True)
sp500_df.drop(columns = ["Date"], inplace = True) 
sp500_df.head()

In [ ]:
# Check Data Types
sp500_df.dtypes

In [ ]:
# Fix Data Types
sp500_df["Close"] = sp500_df["Close"].str.replace("$", "", regex = True) 
sp500_df["Close"] = sp500_df["Close"].astype("float") 
sp500_df.dtypes

In [ ]:
# Calculate Daily Returns  
daily_returns_df = sp500_df.pct_change()
daily_returns_df.head() 

In [ ]:
# Drop nulls
sp500_df = sp500_df.dropna().copy() 
sp500_df.isnull().sum()

In [ ]:
# Rename `Close` Column to be specific to this portfolio.
daily_returns_df = daily_returns_df.rename(columns = { 
    "Close" : "SP500 Daily Returns" 
}) 
daily_returns_df.head()

## Combine Whale, Algorithmic, and S&P 500 Returns

In [ ]:
# Join Whale Returns, Algorithmic Returns, and the S&P 500 Returns into a single DataFrame with columns for each portfolio's returns.
joined_data_cols = pd.concat([whales_df, algo_df, daily_returns_df], axis ='columns', join ='inner')
joined_data_cols.head()

---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [ ]:
# Plot daily returns of all portfolios
joined_data_cols.plot(title = "Daily Returns of Portfolios", figsize = (15,10))

#### Calculate and Plot cumulative returns.

In [ ]:
# Calculate cumulative returns of all portfolios
cumulative_returns = (1 + joined_data_cols).cumprod()
# Plot cumulative returns
cumulative_returns.plot(title = "Cumulative Returns of Portfolios", figsize = (15,10))

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

### Create a box plot for each portfolio


In [ ]:
# Box plot to visually show risk
joined_data_cols.plot(kind = "box", figsize = (20, 7))

### Calculate Standard Deviations

In [ ]:
# Calculate the daily standard deviations of all portfolios
joined_data_std = joined_data_cols.std().sort_values(ascending = False)
joined_data_std

### Determine which portfolios are riskier than the S&P 500

In [ ]:
# Determine which portfolios are riskier than the S&P 500
is_portfolios_riskier_sp500 = [joined_data_std["SP500 Daily Returns"] < joined_data_std]
is_portfolios_riskier_sp500

""" Berkshire Hathaway INC and Tiger Global Management LLC are riskier than the S&P 500 """ 

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
annualized_std = joined_data_std * np.sqrt(252)
annualized_std.sort_values(ascending = False) 

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for all portfolios using a 21-day window
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P 500
3. Choose one portfolio, then calculate and plot the 60-day rolling beta between it and the S&P 500

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [ ]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window
joined_data_cols.rolling(window = 21).std().plot(title = "21 Day Rolling STD", figsize = (25,10))

### Calculate and plot the correlation

In [ ]:
# Calculate the correlation
correlation = joined_data_cols.corr()

# Display correlation matrix
sns.heatmap(correlation, vmin = -1, vmax = 1) 

### Calculate and Plot Beta for a chosen portfolio and the S&P 500

In [ ]:
# Calculate covariance of a single portfolio
algo1_rolling_covariance = joined_data_cols["Algo 1"].rolling(window = 60).cov(joined_data_cols["SP500 Daily Returns"]) 

# Calculate variance of S&P 500
sp500_rolling_variance = joined_data_cols["SP500 Daily Returns"].rolling(window = 60).var() 

# Computing beta
algo1_rolling_beta = algo1_rolling_covariance / sp500_rolling_variance

# Plot beta trend
algo1_rolling_beta.plot(title = "Algo 1 Rolling 60-Day Beta compared to the SP500", figsize = (20,10))

## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half life for each portfolio, using standard deviation (`std`) as the metric of interest.

In [ ]:
# Use `ewm` to calculate the rolling window
ewm_portfolios = joined_data_cols.ewm(span = 21, adjust = False).mean() 
(1 + ewm_portfolios).cumprod().plot(figsize = (20,10))

---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Annualized Sharpe Ratios
sharpe_ratio = (joined_data_cols.mean() * 252) / (joined_data_cols.std() * np.sqrt(252))
sharpe_ratio 

In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratio.plot(kind = "bar", title = "Sharpe Ratios of Portfolios") 

### Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

##### --- Algo 1 outperformed the whale portfolios [Berkshire Hathaway, Soros Fund Management, Paulson & CO INC, Tiger Global Management] and the SP500 daily returns. 
##### --- Algo 2 outperformed the whale portfolios [Soros Fund Management, Paulson & CO INC, Tiger Global Management] and the SP500 daily returns. 

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others
5. Include correlation analysis to determine which stocks (if any) are correlated

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [ ]:
# Reading data from 1st stock (APPLE INC)
aapl_df = pd.read_csv(Path(r"C:\Users\minha\OneDrive\Documents\Pandas_Portfolio_Analysis\Resources\AAPL_price_history.csv"))
aapl_df.set_index(pd.to_datetime(aapl_df["Date"], infer_datetime_format = True), inplace = True)
aapl_df = aapl_df.drop(["Date", "Open", "High", "Low", "Volume"], axis = 1)
aapl_df.rename(columns={"Close":"AAPL"}, inplace = True)
aapl_df.head()

In [ ]:
# Reading data from 2nd stock (AMD Micro Devices INC) 
amd_df = pd.read_csv(Path(r"C:\Users\minha\OneDrive\Documents\Pandas_Portfolio_Analysis\Resources\AMD_price_history.csv"))
amd_df.set_index(pd.to_datetime(amd_df["Date"], infer_datetime_format = True), inplace = True) 
amd_df = amd_df.drop(["Date", "Open", "High", "Low", "Volume"], axis = 1) 
amd_df.rename(columns={"Close":"AMD"}, inplace = True)
amd_df.head()

In [ ]:
# Reading data from 3rd stock (COINBASE Global INC)
coin_df = pd.read_csv(Path(r"C:\Users\minha\OneDrive\Documents\Pandas_Portfolio_Analysis\Resources\COIN_price_history.csv"))
coin_df.set_index(pd.to_datetime(coin_df["Date"], infer_datetime_format = True), inplace = True) 
coin_df = coin_df.drop(["Date", "Open", "High", "Low", "Volume"], axis = 1) 
coin_df.rename(columns={"Close":"COIN"}, inplace = True)
coin_df.head() 

In [ ]:
# Reading data from 4th stock (S&P 60 TSX)
sp500_tsx_df = pd.read_csv(Path(r"C:\Users\minha\OneDrive\Documents\Pandas_Portfolio_Analysis\Resources\TX60_INDEX_price_history.csv"))
sp500_tsx_df.set_index(pd.to_datetime(sp500_tsx_df["Date"], infer_datetime_format = True), inplace = True) 
sp500_tsx_df = sp500_tsx_df.drop(["Date", "Open", "High", "Low", "Volume"], axis = 1) 
sp500_tsx_df.rename(columns={"Close":"S&P/TSX60"}, inplace = True)
sp500_tsx_df.sort_index(ascending = True, inplace = True) 
sp500_tsx_df.head() 

In [ ]:
# Combine all stocks in a single DataFrame
my_portfolio_stocks = pd.concat((aapl_df, amd_df, coin_df), axis = 1, join = "inner") 
my_portfolio_stocks.sort_index(ascending = True, inplace = True) 

# Reset Date index
my_portfolio_stocks.index = my_portfolio_stocks.index.normalize()

my_portfolio_stocks.head() 

In [ ]:
# Calculate daily returns
my_portfolio_returns = my_portfolio_stocks.pct_change()

# Drop NAs
my_portfolio_returns.dropna(inplace = True) 

# Display sample data
my_portfolio_returns.head()

## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return
my_portfolio_weighted_returns  = my_portfolio_returns.dot(weights)

# Display sample data
my_portfolio_weighted_returns.head()

## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Join your returns DataFrame to the original returns DataFrame
joined_portfolio_returns = pd.concat([joined_data_cols, my_portfolio_weighted_returns], axis = 'columns', join ='inner')
joined_portfolio_returns.rename(columns = {0:"my_portfolio"}, inplace = True)

In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)
joined_portfolio_returns.dropna(inplace = True) 
joined_portfolio_returns.head()

##### FIX ::: TO DO ########

## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized `std`


### Calculate and plot rolling `std` with 21-day window

In [ ]:
# Calculate rolling standard deviation

# Plot rolling standard deviation


### Calculate and plot the correlation

In [ ]:
# Calculate and plot the correlation


### Calculate and Plot Rolling 60-day Beta for Your Portfolio compared to the S&P 60 TSX

In [ ]:
# Calculate and plot Beta


### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Calculate Annualized Sharpe Ratios


In [ ]:
# Visualize the sharpe ratios as a bar plot


### How does your portfolio do?

Write your answer here!